In [78]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL to scrape
url = "https://results.eci.gov.in/PcResultGenJune2024/index.htm"

# Make a request to the website
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the div with class "rslt-table table-responsive"
    result_div = soup.find('div', class_='rslt-table table-responsive')

    if result_div:
        # Find the table inside the div
        table = result_div.find('table')

        if table:
            # Extract all rows from the table
            rows = table.find_all('tr')

            # List to store the data from the "Leading" column
            leading_column_data = []

            # Loop through the rows and extract data from the "Leading" column
            for row in rows:
                columns = row.find_all('td')
                if len(columns) > 0:
                    # Assuming the "Leading" column is labeled as such, find the appropriate index
                    header_row = table.find('tr')
                    headers = [th.text.strip() for th in header_row.find_all('th')]
                    
                    if "Leading" in headers:
                        leading_index = headers.index("Leading")
                        leading_column = columns[leading_index]
                        leading_column_data.append(str(leading_column))

            # Join the leading_column_data list into a single string
            leading_column_html = ''.join(leading_column_data)

            # Parse the combined HTML string
            soup = BeautifulSoup(leading_column_html, 'html.parser')
            href_links = [a['href'] for a in soup.find_all('a')]

In [79]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

dataframes = []
for link in href_links:
    url = "https://results.eci.gov.in/PcResultGenJune2024/" + link

    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        result_div = soup.find('div', class_="table-responsive")
        if result_div:
            table = result_div.find('table')
            headers = []
            for th in table.find_all('th'):
                headers.append(th.text.strip())
            rows = []
            for tr in table.find_all('tr')[1:]:  # Skip the header row
                cells = tr.find_all('td')
                row = [cell.text.strip() for cell in cells]
                rows.append(row)
            df = pd.DataFrame(rows, columns=headers)
            dataframes.append(df)

big_df = pd.concat(dataframes, ignore_index=True)


In [80]:
big_df.sample(6)


,S.No,Parliament Constituency,Leading Candidate,Total Votes,Margin
56,57,MANDI(2),KANGNA RANAUT,525691,72696
2,3,Narsapuram(9),BHUPATHI RAJU SRINIVASA VARMA (B.J.P.VARMA),699382,273345
339,1,Kolar(28),M. MALLESH BABU,691481,71388
90,91,CHHINDWARA(16),BUNTY VIVEK SAHU,613853,109006
352,13,Pratapgarh(39),SHIV PAL SINGH PATEL (DR. S P SINGH),362969,54246
458,6,Mumbai North East(28),SANJAY DINA PATIL,377618,25829


In [81]:
big_df['Margin'] = pd.to_numeric(big_df['Margin'], errors='coerce')
sorted_df = big_df.sort_values(by='Margin', ascending=True)
sorted_df = sorted_df.reset_index(drop=True)
sorted_df.to_csv("Final_data.csv")
